In [34]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [111]:
def get_programs(ipeds_id):
    # Send a GET request
    url = f'https://nces.ed.gov/collegenavigator/?s=all&l=92&ic=3&id={ipeds_id}'
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print("Failed to get the webpage.")
    else:
        # Parse the webpage with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the school name
        school_name = soup.find('span', {'class': 'headerlg'}).text.strip()

        # Find the program table
        try:
            program_table = soup.find_all('table', {'class': 'pmtabular'})[0]  # This is the first table with the class "pellgranttable"
        except IndexError:
            print(f"Failed to find the program table for {school_name}, id: {ipeds_id}")
            return None

        # Initialize a dictionary to store the data
        data = {school_name: {}}
        for row in program_table.find_all('tr')[1:]:
            # Split the row into columns
            cols = row.find_all('td')

            # Check if the row has at least two columns
            if len(cols) >= 2:
                # Get the program name 
                program_name = cols[0].text.strip()

                # # Get the completers in the first column, if it's not a '-'
                # completers_col1 = 0
                # if cols[1].text.strip() != '-':
                #     completers_col1 = int(cols[1].text.strip())

                completers = []

                for i in range(1, len(cols)):
                    col_text = cols[i].text.strip()
                    if col_text != '-':
                        col_text_clean = re.sub('\D', '', col_text)  # Remove all non-digit characters
                        try:
                            completers_i = int(col_text_clean)
                        except ValueError:  # Catch conversion errors
                            print(f"Error converting '{col_text_clean}' to integer")
                            completers_i = 0
                    else:
                        completers_i = 0
                    completers.append(completers_i)

                # Calculate the total completers
                total_completers = sum(completers)

                # Add the data to the dictionary
                data[school_name][program_name] = total_completers
            # else:
            #     print(f"Row with insufficient columns: {row}")

        # Convert the dictionary to a DataFrame and print it
        df = pd.DataFrame(data)
        df_t = df.T
        df_t['ipeds_id'] = ipeds_id

    return df_t

In [100]:
# # List to store school dictionaries
# schools = []

# # Start with the first page to get the maximum page number
# url = "https://nces.ed.gov/collegenavigator/?s=all&ct=3&pg=1"

# # Send a GET request
# response = requests.get(url)

# # Parse the webpage with BeautifulSoup
# soup = BeautifulSoup(response.text, 'html.parser')

# # Extract the maximum page number
# page_numbers = []
# pagination_elements = soup.find('div', {'id': 'ctl00_cphCollegeNavBody_ucResultsMain_divPagingControls'}).find_all('strong')
# for element in pagination_elements:
#     page_number = element.text.strip()
#     if page_number.isdigit():
#         page_numbers.append(int(page_number))

# # If no page numbers were found, default to 10 pages
# max_page_number = max(page_numbers) if page_numbers else 10

# # Regular expression pattern to match 'results' followed by any letter
# pattern = re.compile(r'results\w')

# # Loop from the first page to the maximum page number
# for page_num in range(1, 5): #range(1, max_page_number + 1):
#     # Form the URL with the current page number
#     url = f"https://nces.ed.gov/collegenavigator/?s=all&ct=3&pg={page_num}"

#     # Send a GET request
#     response = requests.get(url)

#     # Parse the webpage with BeautifulSoup
#     soup = BeautifulSoup(response.text, 'html.parser')

#     # Extract the schools and IPEDS IDs
#     ipeds_elements = soup.find_all('p', {'class': 'ipeds hoverID'})
#     full_table = BeautifulSoup(str(soup.find_all('table', {'class': 'resultsTable'})[0]))
#     school_elements = full_table.find_all('strong')
#     for ipeds_element, school_element in zip(ipeds_elements, school_elements):
#         school_name = school_element.text
#         ipeds_id = ipeds_element.text.split('|')[0].split(':')[1].strip()
#         schools.append({'School Name': school_name, 'ipedsID': ipeds_id})

# # Convert the list of dictionaries to a DataFrame
# school_id_df = pd.DataFrame(schools)


In [124]:
# List to store ids
ipeds_ids = []

for school_length in range(1, 4):
    for school_type in range(1,4):
        # Start with the first page to get the maximum page number
        url = f"https://nces.ed.gov/collegenavigator/?s=all&ct={school_type}&ic={school_length}&pg=1"

        # Send a GET request
        response = requests.get(url)

        # Parse the webpage with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the maximum page number
        page_numbers = []
        pagination_elements = soup.find('div', {'id': 'ctl00_cphCollegeNavBody_ucResultsMain_divPagingControls'}).find_all('strong')
        for element in pagination_elements:
            page_number = element.text.strip()
            if page_number.isdigit():
                page_numbers.append(int(page_number))

        # If no page numbers were found, default to 10 pages
        max_page_number = max(page_numbers) if page_numbers else 10

        # Regular expression pattern to match 'results' followed by any letter
        pattern = re.compile(r'results\w')


        # Loop from the first page to the maximum page number
        for page_num in range(1, max_page_number + 1):
            # Form the URL with the current page number
            url = f"https://nces.ed.gov/collegenavigator/?s=all&ct={school_type}&ic={school_length}&pg={page_num}"
            # print(url)

            # Send a GET request
            response = requests.get(url)

            # Parse the webpage with BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract the schools and IPEDS IDs
            ipeds_elements = soup.find_all('p', {'class': 'ipeds hoverID'})
            full_table = BeautifulSoup(str(soup.find_all('table', {'class': 'resultsTable'})[0]))
            # school_elements = full_table.find_all('strong')
            for ipeds_element in ipeds_elements:
                ipeds_id = ipeds_element.text.split('|')[0].split(':')[1].strip()
                ipeds_ids.append(ipeds_id)

        print(f"Finished Type {school_type}")
    print(f"Finished Length {school_length}")

Finished Type 1
Finished Type 2
Finished Type 3
Finished Length 1
Finished Type 1
Finished Type 2
Finished Type 3
Finished Length 2
Finished Type 1
Finished Type 2
Finished Type 3
Finished Length 3


In [125]:
print(len(ipeds_ids))

3498


In [126]:
total_df = pd.DataFrame()

counter = 0
for ipeds_id in ipeds_ids:
    total_df = pd.concat([total_df, get_programs(ipeds_id)])
    counter += 1
    if counter % 100 == 0:
        print(f"Processed {counter} IDs.")

Failed to find the program table for Arizona State University - Cochise, id: 10415109
Failed to find the program table for Arizona State University - Lake Havasu, id: 10415105
Failed to find the program table for Arizona State University - Mesa City Center, id: 10415114
Failed to find the program table for Arizona State University - Northeastern Arizona, id: 10415113
Failed to find the program table for Arizona State University - Pima, id: 10415111
Failed to find the program table for Arizona State University - Pinal, id: 10415108
Failed to find the program table for Arizona State University - The Gila Valley, id: 10415106
Failed to find the program table for Arizona State University - Tucson, id: 10415104
Failed to find the program table for Arizona State University - Washington, D.C., id: 10415112
Failed to find the program table for Arizona State University - Yavapai, id: 10415110
Failed to find the program table for Arizona State University - Yuma, id: 10415107
Failed to find the p

In [132]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [130]:
total_df = total_df.drop(columns = ['Category total', 'Grand total'])

In [128]:
# total_raw = total_df

In [133]:
total_df

American Indian/Native American Studies  \
Aaniiih Nakoda College                                                                  1.0   
Abraham Baldwin Agricultural College                                                    NaN   
Adams State University                                                                  NaN   
Air Force Institute of Technology-Graduate Scho...                                      NaN   
Alabama A & M University                                                                NaN   
...                                                                                     ...   
Empire Beauty School-Thornton                                                           NaN   
Empire Beauty School-Tucson                                                             NaN   
Empire Beauty School-Union                                                              NaN   
Empire Beauty School-Vernon Hills                                                       NaN   
Empire Beauty School-Virginia Beach                                                     NaN   

                                                    Behavioral Aspects of Health  \
Aaniiih Nakoda College                                                       2.0   
Abraham Baldwin Agricultural College                                         NaN   
Adams State University                                                       NaN   
Air Force Institute of Technology-Graduate Scho...                           NaN   
Alabama A & M University                                                     NaN   
...                                                                          ...   
Empire Beauty School-Thornton                                                NaN   
Empire Beauty School-Tucson                                                  NaN   
Empire Beauty School-Union                                                   NaN   
Empire Beauty School-Vernon Hills                                            NaN   
Empire Beauty School-Virginia Beach                                          NaN   

                                                    Business/Commerce, General  \
Aaniiih Nakoda College                                                     3.0   
Abraham Baldwin Agricultural College                                      42.0   
Adams State University                                                   120.0   
Air Force Institute of Technology-Graduate Scho...                         NaN   
Alabama A & M University                                                   NaN   
...                                                                        ...   
Empire Beauty School-Thornton                                              NaN   
Empire Beauty School-Tucson                                                NaN   
Empire Beauty School-Union                                                 NaN   
Empire Beauty School-Vernon Hills                                          NaN   
Empire Beauty School-Virginia Beach                                        NaN   

                                                    Business/Office Automation/Technology/Data Entry  \
Aaniiih Nakoda College                                                                           3.0   
Abraham Baldwin Agricultural College                                                             NaN   
Adams State University                                                                           NaN   
Air Force Institute of Technology-Graduate Scho...                                               NaN   
Alabama A & M University                                                                         NaN   
...                                                                                              ...   
Empire Beauty School-Thornton                                                                    NaN   
Empire Beauty School-Tucson                                                          

In [136]:
import json
import gspread
from gspread_dataframe import set_with_dataframe

In [137]:
credentials = json.load(open("/Users/sophiechance/Documents/CourseKey/syncs/google_credentials.json"))
gc = gspread.service_account_from_dict(credentials)

In [139]:
ipeds_programs_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Vndkl25DmFbJUoDH1MONITRjgHViRAYxLPaaFr-Tndg/edit#gid=0')

In [143]:
total_df = total_df.reset_index().rename(columns = {'index': 'School Name'})

In [144]:
ipeds_programs_wb.worksheet('IPEDS Data').clear()
set_with_dataframe(ipeds_programs_wb.worksheet('IPEDS Data'), total_df)

In [159]:
programs_df = pd.DataFrame(total_df.columns).rename(columns = {0: 'Program Name'}).query('`Program Name` != "School Name" and `Program Name` != "ipeds_id"')

In [160]:
ipeds_programs_wb.worksheet('Program List').clear()
set_with_dataframe(ipeds_programs_wb.worksheet('Program List'), programs_df)